In [1]:
import os
import sys
import numpy as np
import pandas as pd
import cv2 as cv
from matplotlib.image import imread
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from sklearn import model_selection
from os import listdir
from os.path import isfile, join
import tensorflow as tf
from skimage.io import imread
import cv2
import re

C:\Users\Mary\AppData\Local\Programs\Python\Python37\lib\site-packages\skimage\io\manage_plugins.py:23: UserWarning: Your installed pillow version is < 7.1.0. Several security issues (CVE-2020-11538, CVE-2020-10379, CVE-2020-10994, CVE-2020-10177) have been fixed in pillow 7.1.0 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [31]:
import PIL
import PIL.Image
import tensorflow as tf
from tensorflow.keras.applications.resnet_v2 import ResNet152V2

In [3]:
dir = os.path.dirname
src_path = dir(os.path.abspath(''))

In [4]:
path = src_path + os.sep + "src" +  os.sep + "utils"  

In [5]:
sys.path.append(path)
import folders_tb as fol 

In [6]:
path_train = src_path + os.sep + "data" +  os.sep + "train"  +  os.sep + '**'  +  os.sep + '**.jpg'
path_train

'c:\\Users\\Mary\\Desktop\\BootCamp\\Python\\MaryC-MezaR\\data_science_bootcamp_2021\\Machine_Learning_project\\data\\train\\**\\**.jpg'

In [12]:
train = fol.make_imag_df(path_train)

In [15]:
train = fol.creat_colum (train)

In [16]:
path_path = src_path + os.sep + "data" + os.sep + "train"  +  os.sep 

In [17]:
train_set = fol.import_imag(path_path, train)

In [18]:
path_test = src_path + os.sep + "data" +  os.sep + "test"  +  os.sep + '**'  +  os.sep + '**.jpg'
path_test

'c:\\Users\\Mary\\Desktop\\BootCamp\\Python\\MaryC-MezaR\\data_science_bootcamp_2021\\Machine_Learning_project\\data\\test\\**\\**.jpg'

In [19]:
test = fol.make_imag_df(path_test)

In [20]:
test = fol.creat_colum (test)

In [21]:
path_path_2 = src_path + os.sep + "data" + os.sep + "test"  +  os.sep 

In [22]:
test_set = fol.import_imag(path_path_2, test)
test_set

,set,name,id,img,path,label,nombre_img
0,test,battery,battery101,battery101.jpg,battery/battery101.jpg,0,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
1,test,battery,battery108,battery108.jpg,battery/battery108.jpg,0,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
2,test,battery,battery122,battery122.jpg,battery/battery122.jpg,0,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
3,test,battery,battery132,battery132.jpg,battery/battery132.jpg,0,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
4,test,battery,battery14,battery14.jpg,battery/battery14.jpg,0,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
...,...,...,...,...,...,...,...
1198,test,plastic,plastic864,plastic864.jpg,plastic/plastic864.jpg,6,"[[[182, 191, 200], [185, 194, 203], [184, 192,..."
1199,test,plastic,plastic90,plastic90.jpg,plastic/plastic90.jpg,6,"[[[200, 221, 236], [200, 221, 236], [200, 221,..."
1200,test,plastic,plastic93,plastic93.jpg,plastic/plastic93.jpg,6,"[[[216, 203, 201], [215, 202, 200], [214, 201,..."
1201,test,plastic,plastic95,plastic95.jpg,plastic/plastic95.jpg,6,"[[[255, 242, 240], [255, 242, 240], [255, 242,..."


In [23]:
X_train = np.stack(np.array(train_set["nombre_img"]))
y_train = np.array(train_set["label"])
print(X_train.shape)
print(y_train.shape)

(4803, 224, 224, 3)
(4803,)


In [24]:
X_test = np.stack(np.array(test_set["nombre_img"]))
y_test = np.array(test_set["label"])
print(X_test.shape)
print(y_test.shape)

(1203, 224, 224, 3)
(1203,)


In [ ]:
X_train = X_train.reshape(4803, 224, 224, 3)
X_train.shape

In [25]:
X_val = X_train[-900:]
y_val = y_train[-900:]
X_train = X_train[:-900]
y_train = y_train[:-900]

In [26]:
X_test = X_test.reshape(1203, 224, 224, 3)
y_test = y_test.reshape(1203,1)

In [27]:
datagen = ImageDataGenerator(rescale=1./255)

In [28]:
datagen.fit(X_train)

In [29]:
datagen.fit(X_test)

In [30]:
datagen.fit(X_val)

In [ ]:
## Entrenar el modelo

In [34]:
img_height = 224
img_width  = 224

base_model_resnet = ResNet152V2(input_shape = (img_height, img_width,3),
                        include_top = False,
                        weights = 'imagenet')

for layer in base_model_resnet.layers:
    layer.trainable = False

# Se añaden dios capas adicionales para una predicción binaria
x0 = layers.Flatten()(base_model_resnet.output)
x1 = layers.Dense(32, activation='relu')(x0)   
x2 = layers.Dropout(0.5)(x1) 
x3 = layers.Dense(7 , activation='softmax')(x2)

model_resnet = tf.keras.models.Model(base_model_resnet.input, x3)

model_resnet.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics = ['acc'])

RN152_history = model_resnet.fit(X_train, y_train,
                       validation_data = (X_val, y_val),
                       epochs = 10,
                       verbose=1) 

Epoch 1/10
122/122 [==============================] - 713s 6s/step - loss: 236.9650 - acc: 0.1845 - val_loss: 2.0317 - val_acc: 0.0000e+00
Epoch 2/10
122/122 [==============================] - 741s 6s/step - loss: 1.9223 - acc: 0.2016 - val_loss: 2.1235 - val_acc: 0.0000e+00
Epoch 3/10
122/122 [==============================] - 746s 6s/step - loss: 1.8893 - acc: 0.2019 - val_loss: 2.2121 - val_acc: 0.0000e+00
Epoch 4/10
122/122 [==============================] - 722s 6s/step - loss: 1.8738 - acc: 0.2019 - val_loss: 2.2978 - val_acc: 0.0000e+00
Epoch 5/10
122/122 [==============================] - 810s 7s/step - loss: 1.8614 - acc: 0.2019 - val_loss: 2.3795 - val_acc: 0.0000e+00
Epoch 6/10
122/122 [==============================] - 764s 6s/step - loss: 1.8512 - acc: 0.2019 - val_loss: 2.4575 - val_acc: 0.0000e+00
Epoch 7/10
122/122 [==============================] - 716s 6s/step - loss: 1.8427 - acc: 0.2019 - val_loss: 2.5331 - val_acc: 0.0000e+00
Epoch 8/10
122/122 [===================

In [39]:
test_loss, test_acc = model_resnet.evaluate(X_test,  y_test, verbose=1)
print('\nTest accuracy:', test_acc, "\nLoss:", test_loss)

38/38 [==============================] - 170s 4s/step - loss: 2.0494 - acc: 0.1638

Test accuracy: 0.16375727951526642 
Loss: 2.0494203567504883


In [35]:
path_save = path = src_path +  os.sep + "models" + os.sep

In [37]:
model_resnet.save(path_save + "modelresnetV2.h5")

C:\Users\Mary\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [ ]:
model_resnet.save(path_save + "modelresnetV2.h5")